In [1]:
import json
!pip install -Uq requests
import requests
!mkdir -p data

# Using an API

Learning outcomes

- difference between API & webscraping
- what JSON is (and why it's like a Python `dict`)
- how to properly handle files in Python
- what a REST API is
- how to use the `requests` library

## API versus web-scraping

**Both are ways to sample data from the internet**

API
- structured
- provided as a service (you are talking to a server via a REST API)
- limited data / rate limits / paid / require auth (sometimes)
- most will give back JSON (maybe XML or CSV)

Web scraping
- less structure
- parsing HTML meant for your browser

Neither is better than the other

- API developer can limit what data is accessible through the API
- API developer can not maintain the API
- website page can change HTML structure
- website page can have dynamic (Javascript) content that requires execution (usually done by the browser) before the correct HTML is available

Much of the work in using an API is figuring out how to properly construct URLs for `GET` requests
- requires looking at their documentation (& ideally a Python example!)

## Where to find APIs

- [ProgrammableWeb](https://www.programmableweb.com/apis/directory) - a collection of available API's
- For the *Developer* or *For Developers* documentation on your favourite website
- [public-apis/public-apis](https://github.com/public-apis/public-apis)

## Using APIs

Most APIs require authentication

- so they API developer knows who you are
- can charge you
- can limit access
- commonly via key or OAuth (both of which may be free)

All the APIs we use here are unauthenticated - this is to avoid the time of you all signing up

If your app requries authentication, it's usually done by passing in your credentials into the request (as a header)

```python
response = requests.get(url, auth=auth)
```

## JSON strings

JSON (JavaScript Object Notation) is a:
- lightweight data-interchange format (text)
- easy for humans to read and write 
- easy for machines to parse and generate
- based on key, value pairs

You can think of the Python `dict` as JSON like:

In [2]:
data = {'name': 'alan-turing'}
data

{'name': 'alan-turing'}

But true JSON is just a string (only text).  We can use `json.dumps` from the standard library to turn the `dict` into a JSON string:

In [3]:
data = json.dumps(data)
data

'{"name": "alan-turing"}'

In [4]:
type(data)

str

We can then use `json.loads` to turn this string back into a `dict`:

In [5]:
json.loads(data)

{'name': 'alan-turing'}

## Opening, reading & writing to files

### Reading from a file

We open files using the Python `open` builtin function, followed by a `read`:

In [6]:
open('./readme.md', 'r').read()[:100]

'# Introduction to Python at Data Science Retreat \n\n## Topics:\n1. Introduction: \n  - Data Types \n  - '

(Note the `./path` - the `.` refers to the current working directory.  It's not straightforward to know where this is - is it where the notebook is, or where the notebook server is running?

If we wanted to read the file as separate lines, we could use `readlines()` (note we would still need to manually strip off the `\n` characters later)

(Note `\n` is used as a newline indicator in text files - you never see it because your editor interprets it as a line break :)

In [7]:
open('./readme.md', 'r').readlines()[:8]

['# Introduction to Python at Data Science Retreat \n',
 '\n',
 '## Topics:\n',
 '1. Introduction: \n',
 '  - Data Types \n',
 '  - Conditionals \n',
 '  - For Loops\n',
 '  - PEP8 Coding Standards\n']

## Using `open`

`open(path, mode)`

Common values for the mode:
- `r` read
- `rb` read binary
- `w+` write (`+` to create file if it doesn't exist)
- `a` append

Note there are options for both reading & writing - we actually use `open` for both reading & writing.

We open a file using the Python builtin `open`, which is then followed by either a read or write stage

- open the file
- read the file OR write to the file

Notice that the file is read in as a single string, with the newline character `\n` separating lines

- this is how all text files are structured
- your editor does the line splitting for you

### Writing to a file (without context management)

We can write to a new file using the same `open` builtin

- open the file
- write to the file

In [8]:
open('./data/output.data', 'w').write('We make this file to show how not to do it\n')

43

Note that we can do the same file write by explicitly assiging the file object to a variable (note the `a` to append)

In [9]:
fi = open('./data/output.data', 'a')
fi.write('We make this file slightly differently to show how not to do it\n')

64

The issue with the code above is that we aren't closing the file - we can fix this by intentionally closing the file.  

One way to do this is to use `.close()` when we are done:

In [10]:
fi = open('./data/output.data', 'a')
fi.write('This time we close the file manually\n')
fi.close()

This requires us to remember to close (also an additional line).

### Reading files with context management

The Pythonic way of handling opening & closing of files is context management:

In [11]:
with open('./readme.md', 'r') as fi:
    data = fi.read()

The difference with using context management (which essentially just means using the `with` statement) is that it closes the file automatically when finished writing/reading.

### Writing to a file with context management

Using the context management syntax, we can save our `data` dict as JSON, using `json.dump` to write the dict to a file:

In [12]:
data = {'name': 'alan turing'}
with open('./data/output.json', 'w') as fi:
    json.dump(data, fi)

Let's check it worked by loading the file, here using `json.load` to load from the file object:

In [13]:
with open('./data/output.json', 'r') as fi:
    data = json.load(fi)
data

{'name': 'alan turing'}

## REST APIs

[REST - Wiki](https://en.wikipedia.org/wiki/Representational_state_transfer)

REST is a set of constraints that allow **stateless communication of text data on the internet**

- REST = REpresentational State Transfer
- API = Application Programming Interface

REST
- communication of resources (located at URLs / URIs)
- requests for a resource are responded to with a text payload (HTML, JSON etc)
- these requests are made using HTTP (determines how messages are formatted, what actions (methods) can be taken)
- common HTTP methods are `GET` and `POST`

HTTP methods
- GET - retrieve information about the REST API resource
- POST - create a REST API resource
- PUT - update a REST API resource
- DELETE - delete a REST API resource or related component

RESTful APIs enable you to develop any kind of web application having all possible CRUD (create, retrieve, update, delete) operations

- can do anything we would want to do with a database

*Further reading*
- [Web Architecture 101](https://medium.com/storyblocks-engineering/web-architecture-101-a3224e126947) for more detail on how the web works

## Example - sunrise API

Docs - https://sunrise-sunset.org/api

First we need to form the url
- use `?` to separate the API server name from the parameters for our request
- use `&` to separate the parameters from each other
- use `+` instead of space in the parameter

In [14]:
# getting sunrise & sunset for Berlin today
res = requests.get("https://api.sunrise-sunset.org/json?lat=52.5200&lng=13.4050")
data = res.json()
data

{'results': {'sunrise': '7:10:45 AM',
  'sunset': '3:17:49 PM',
  'solar_noon': '11:14:17 AM',
  'day_length': '08:07:04',
  'civil_twilight_begin': '6:32:45 AM',
  'civil_twilight_end': '3:55:49 PM',
  'nautical_twilight_begin': '5:49:26 AM',
  'nautical_twilight_end': '4:39:07 PM',
  'astronomical_twilight_begin': '5:08:17 AM',
  'astronomical_twilight_end': '5:20:16 PM'},
 'status': 'OK'}

In [40]:
res = requests.get("https://api.sunrise-sunset.org/json?lat=40.4178273&lng=-3.7240931")
data = res.json()
data

{'results': {'sunrise': '7:35:41 AM',
  'sunset': '5:09:57 PM',
  'solar_noon': '12:22:49 PM',
  'day_length': '09:34:16',
  'civil_twilight_begin': '7:07:06 AM',
  'civil_twilight_end': '5:38:31 PM',
  'nautical_twilight_begin': '6:33:19 AM',
  'nautical_twilight_end': '6:12:18 PM',
  'astronomical_twilight_begin': '6:00:31 AM',
  'astronomical_twilight_end': '6:45:07 PM'},
 'status': 'OK'}

This response is JSON - `requests.json()` turns it into a `dict`:

In [41]:
type(data)

dict

It's common to have a top level heirarchy to dig through to get the data:

In [42]:
data.keys()

dict_keys(['results', 'status'])

Here the interesting stuff is in `results`:

In [17]:
data['results']

{'sunrise': '7:10:45 AM',
 'sunset': '3:17:49 PM',
 'solar_noon': '11:14:17 AM',
 'day_length': '08:07:04',
 'civil_twilight_begin': '6:32:45 AM',
 'civil_twilight_end': '3:55:49 PM',
 'nautical_twilight_begin': '5:49:26 AM',
 'nautical_twilight_end': '4:39:07 PM',
 'astronomical_twilight_begin': '5:08:17 AM',
 'astronomical_twilight_end': '5:20:16 PM'}

### Exercise: 
Save the parameter `sunrise` as a string value by itself in a variable called `sunrise_time`

In [60]:
data["results"]["sunrise"]

dict

## Example - Chronicling America API

Docs - https://chroniclingamerica.loc.gov/about/api/ 

In [61]:
term = "germany"
fmt = "json"
url = f"https://chroniclingamerica.loc.gov/search/pages/results/?proxtext={term}&format={fmt}"
url

'https://chroniclingamerica.loc.gov/search/pages/results/?proxtext=germany&format=json'

We use the `requests` HTTP library to perform a `GET` request:

In [19]:
response = requests.get(url)

In [20]:
response

<Response [200]>

## HTTP response

What we recieved above is an *HTTP response**

[HTTP Response - Wiki](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Response_message)

The response message consists of the following:

- a status line which includes the status code and reason message (e.g., HTTP/1.1 200 OK, which indicates that the client's request succeeded)
- response header fields (e.g., Content-Type: text/html)
- an optional message body

## What can we do with this HTTP response in `requests`?

The Python builtin `dir` gives us all the attributes & methods of a Python object.

This also includes all the `__` dunder (literally double-under) methods) - which we filter out using a list comprehension.

In [ ]:
[item for item in dir(response) if '__' not in item]

We can get the HTTP status code (used to communicate things like everything OK (200), stop making requests etc - see [List of HTTP status codes - Wiki](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)):

In [22]:
response.status_code

200

The HTTP response headers:

In [ ]:
response.headers

And the response body:

In [ ]:
response.text[:1000]

In [25]:
import json
data = json.loads(response.text)

In [ ]:
data

We can access the keys of the dictionary:

In [27]:
data.keys()

dict_keys(['totalItems', 'endIndex', 'startIndex', 'itemsPerPage', 'items'])

We can access the values using the square bracket indexing with a key:

In [28]:
data['totalItems']

2334262

While JSON is a simple text format, it can become complex due to

- nesting (JSON inside JSON)
- lists of JSON

An example is our `items`, which has been parsed as a Python `list`:

In [29]:
type(data['items'])

list

`items` is a list of dicts:

In [30]:
item = data['items'][0]
item.keys()

dict_keys(['sequence', 'county', 'edition', 'frequency', 'id', 'subject', 'city', 'date', 'title', 'end_year', 'note', 'state', 'section_label', 'type', 'place_of_publication', 'start_year', 'edition_label', 'publisher', 'language', 'alt_title', 'lccn', 'country', 'ocr_eng', 'batch', 'title_normal', 'url', 'place', 'page'])

We can iterate over both the keys and values as a pair using `items()`.

Below we use a check  a quick check that the value isn't too long before printing:

In [ ]:
from collections.abc import Iterable

for k, v in item.items():
    if isinstance(v, Iterable) and len(v) < 100:
        print(f'{k}: {v}')

Let's finish this exercise by only taking articles that appear between two years, and save those to disk.

Normally you would apply this kind of filtering in the API request - we are going to filter in memory.

First we need a bit of data cleaning of our date, which is an integer representation of time (but is a `str`):

In [32]:
item['date']

'19440912'

Here use `strptime` to convert the integer into a proper datetime:
- ([Python's strftime directives](http://strftime.org/) is very useful!)

In [33]:
from datetime import datetime as dt
dt.strptime(item['date'], "%Y%m%d")

datetime.datetime(1944, 9, 12, 0, 0)

Now let's put this data cleaning & filtering into a pipeline:

In [34]:
term = "germany"
fmt = "json"
url = f"https://chroniclingamerica.loc.gov/search/pages/results/?proxtext={term}&format={fmt}"
res = requests.get(url)
data = res.json()
items = data['items']

start = 1900
extract = []
for item in items:
    item['date'] = dt.strptime(item['date'], "%Y%m%d")
    
    if item['date'].year > start:
        extract.append(item)
        
len(extract)

20

We have a list of dictionaries, which plays very nice with `pandas`:

In [35]:
import pandas as pd
df = pd.DataFrame(extract)
df.head(2)

,sequence,county,edition,frequency,id,subject,city,date,title,end_year,...,language,alt_title,lccn,country,ocr_eng,batch,title_normal,url,place,page
0,11,[Wayne],Home ed.,Daily,/lccn/sn88063294/1944-09-12/ed-1/seq-11/,"[Detroit (Mich.)--Newspapers., Michigan--Detro...",[Detroit],1944-09-12,Detroit evening times.,1958,...,[English],"[Detroit Sunday times, Detroit times]",sn88063294,Michigan,Where the Battle of Germany Will Be Fought\n|D...,mimtptc_holly_ver03,detroit evening times.,https://chroniclingamerica.loc.gov/lccn/sn8806...,[Michigan--Wayne--Detroit],
1,16,[San Francisco],None,Daily,/lccn/sn85066387/1908-11-08/ed-1/seq-16/,[California--San Francisco Bay Area.--fast--(O...,[San Francisco],1908-11-08,The San Francisco call. [volume],1913,...,[English],"[Call, Call-chronicle-examiner, Sunday call]",sn85066387,California,"JACK— he w.45 11111^^ Germany,\nI^ll/ riFSIIOV...",curiv_llano_ver01,san francisco call.,https://chroniclingamerica.loc.gov/lccn/sn8506...,[California--San Francisco--San Francisco],16


## Example - downloading images

We can also `requests` to download things other than text - such as images.

Below we do a requests and see we get back a binary string:

In [ ]:
url = 'https://www.google.com/images/branding/googlelogo/2x/googlelogo_color_272x92dp.png'
res = requests.get(url)
res.text[:100]

We can use context management to dump the contents of the binary string into a file:

In [37]:
with open('./data/google-logo.png', 'wb') as fi:
    fi.write(res.content)

We now have the Google logo locally (you may need to re-run this cell)

![](./data/google-logo.png)

## Exercise - earthquake API 

Let's as a group write a program to get data from the USGS Earthquake Catalog - [documentation](https://earthquake.usgs.gov/fdsnws/event/1/#methods)

Steps:
- Make a folder (from your current working directory) to hold the earthquake data 
- Investigate the data from the response 
- Save each earthquake as its own JSON in the folder you created (hint - you will need to decide on a name for each earthquake JSON) 

In [70]:
start = "2014-01-01"
end = "2014-01-02"
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start}&endtime={end}"

import os
os.makedirs('./data/earthquakes', exist_ok=True)
os.makedirs(f'./data/earthquakes/{start}', exist_ok=True)

In [ ]:
# get data
res = requests.get(url)
data = res.json()


In [ ]:
data["features"]

In [125]:
print(pd.to_datetime(1388620296020, unit='ms').date)


<built-in method date of Timestamp object at 0x7f878e0a4750>


In [ ]:
for i in data["features"]:
  print(i['properties']['title'])

In [112]:
for i in data["features"]:
  e_name = i['properties']['title']
  with open(f"{e_name}.json", "w") as file:
    json.dump(i, file)

## Exercise - Wikipedia API

Now for an open-ended exercise for you! Your task is to:
- create a database of countries
- in a folder called `countries` (you will need to make the folder)
- each country in it's own folder
- start with Germany

V1 of your program should:
- save the url you use to request the data
- save the title
- save the `line` parameter of each section (`data['parse']['sections']`)
- save all in a single JSON

V2 of your program should also:
- save all '.png' & '.jpg' images as images, with the url as the image name
- save all external links as CSV

Much of the work will be understanding how the Wikipedia API works - useful resources are below:
- [Main API page](https://www.mediawiki.org/wiki/API:Main_page)
- [What the actions are](https://www.mediawiki.org/w/api.php)
- [Python examples](https://github.com/wikimedia/mediawiki-api-demos/tree/master/python)

Please also feel free to work on another API - happy to assist you with this as well :)

In [ ]:
import requests
import json
import wikipedia


# global vars
countries = ["Vatican City", "Monaco", "Nauru", "Tuvalu", "San Marino", "Liechtenstein", "Maldives", "Malta"]
title = countries[0]
url = f"https://en.wikipedia.org/w/api.php?action=parse&page={title}&format=json"


for i in countries:
    # static local vars
    title = i
    os.mkdir(i) # create directory
    filename = f"{i}/{i}.json" # .json file name
    res = requests.get(url)
    data = res.json()
    dict_url = {'url': url}
    dict_title = {'title': data['parse']['title']}
    
    # get lines
    lines = []
    for i in data['parse']['sections']:
        line = i['line']
        lines.append(line)

    # build dictionary
    dict_lines = {'lines':lines} # convert to dictionary
    dict_images = {'images': data['parse']['images']} # dict with list of images
    dict_combined = {**dict_url, **dict_title, **dict_lines, **dict_images}
    
    # store json in folder
    with open(filename, "w") as f:
        json.dump(dict_combined, f)